![Henry Logo](https://www.soyhenry.com/_next/static/media/HenryLogo.bb57fd6f.svg)

# Introducción a las bases de datos vectoriales 
## Clase #2 :Transformers

In [1]:
import os
from openai import OpenAI
import pandas as pd
import numpy as np
import tiktoken

In [2]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

MODEL_NAME = "gpt-4o-mini"

In [3]:
text = "Nunca es demasiado tarde para ser sabio."
encoding = tiktoken.encoding_for_model(MODEL_NAME)
# Convertir texto a tokens (IDs)
token_ids = encoding.encode(text)
print(f"IDs de Tokens: {token_ids}")

# Ver los tokens en texto (decodificando uno por uno para inspeccionar)
tokens_text = [encoding.decode_single_token_bytes(token).decode('utf-8') for token in token_ids]
print(f"Tokens de texto: {tokens_text}")

# Mostrar tabla comparativa (visualización típica del Cap 3)
df_tokens = pd.DataFrame({"Token ID": token_ids, "Token Text": tokens_text})
print(df_tokens)

IDs de Tokens: [170149, 878, 55219, 26745, 1209, 1334, 6928, 726, 13]
Tokens de texto: ['Nunca', ' es', ' demasiado', ' tarde', ' para', ' ser', ' sab', 'io', '.']
   Token ID  Token Text
0    170149       Nunca
1       878          es
2     55219   demasiado
3     26745       tarde
4      1209        para
5      1334         ser
6      6928         sab
7       726          io
8        13           .


In [6]:
response = client.embeddings.create(
    input=text,
    model="text-embedding-3-small"
)

embedding_vector = response.data[0].embedding
print(f"Dimensión del vector: {len(embedding_vector)}") 
print(f"Primeros 15 valores: {embedding_vector[:15]}")

Dimensión del vector: 1536
Primeros 15 valores: [0.04944195970892906, 0.0063703227788209915, -0.023048294708132744, 0.02860943041741848, 0.0323023721575737, 0.005335756577551365, -0.002712682355195284, 0.03712491691112518, -0.039753422141075134, 0.006033613346517086, 0.01627066358923912, -0.006707032211124897, 0.020007049664855003, -0.018019378185272217, 0.026958467438817024]


In [8]:
prompt = "La capital de Argentina es "

response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[{"role": "user", "content": prompt}],
    max_tokens=15,
    logprobs=True,      # <--- Esto habilita la inspección
    top_logprobs=5      # <--- Ver las top 5 opciones para cada token
)

# Extraer el contenido generado
generated_content = response.choices[0].message.content
print(f"Prompt: {prompt}")
print(f"Completado: {generated_content}\n")

Prompt: La capital de Argentina es 
Completado: La capital de Argentina es Buenos Aires.



In [9]:
print("--- Inspección de Probabilidades (Looking Inside) ---")
for i, token_data in enumerate(response.choices[0].logprobs.content):
    token = token_data.token
    prob = np.exp(token_data.logprob) * 100  # Convertir logprob a % real
    
    print(f"\nPaso {i+1}: Token elegido -> '{token}' ({prob:.2f}% confianza)")
    
    print("   Otras opciones consideradas:")
    for top_k in token_data.top_logprobs:
        top_prob = np.exp(top_k.logprob) * 100
        print(f"     - '{top_k.token}': {top_prob:.2f}%")

--- Inspección de Probabilidades (Looking Inside) ---

Paso 1: Token elegido -> 'La' (100.00% confianza)
   Otras opciones consideradas:
     - 'La': 100.00%
     - 'Buenos': 0.00%
     - 'la': 0.00%
     - ' La': 0.00%
     - 'El': 0.00%

Paso 2: Token elegido -> ' capital' (100.00% confianza)
   Otras opciones consideradas:
     - ' capital': 100.00%
     - ' Capital': 0.00%
     - ' capit': 0.00%
     - 'capital': 0.00%
     - ' ciudad': 0.00%

Paso 3: Token elegido -> ' de' (100.00% confianza)
   Otras opciones consideradas:
     - ' de': 100.00%
     - ' del': 0.00%
     - 'de': 0.00%
     - ' of': 0.00%
     - ' Argentina': 0.00%

Paso 4: Token elegido -> ' Argentina' (100.00% confianza)
   Otras opciones consideradas:
     - ' Argentina': 100.00%
     - 'Argentina': 0.00%
     - ' argentina': 0.00%
     - ' Arg': 0.00%
     - ' Argent': 0.00%

Paso 5: Token elegido -> ' es' (100.00% confianza)
   Otras opciones consideradas:
     - ' es': 100.00%
     - ' Es': 0.00%
     - ' is'